In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from data.transaction_data_generator import generate_transaction_data
from preparation.data_preparer import prepare_data
from visualization.data_visualizer import visualize_data
from model.model_creator import create_model
from model.solution_parser import parse_solution
from pulp import *
from visualization.solution_data_visualizer import visualize_solution_data, plot_solution_preview
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
style = """
    <style>
        .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
    </style>
    """
display(HTML(style))

In [4]:
importance_factors = {"grocery":1, "fashion":2, "shopping":3, "travel":10, "rent": 1000, "unknown":10, "income":0}
transactions = generate_transaction_data()
prepare_data(transactions)
year = 2019
selected_year = transactions[transactions['year']==year].copy()
selected_year["importance"] = selected_year['type'].map(importance_factors)

In [5]:
text = widgets.HTML(
    value="<b>Budget Optimization</b>: Select inputs",
)

savings_slider = widgets.FloatSlider( min=0,
    max=0.8,
    step=0.05,
    description='Savings:',
    value=0.3)
grocery_slider = widgets.FloatSlider( min=0,
    max=500.0,
    step=5.00,
    description='Grocery:',
    value=50.0)
display(text, savings_slider,grocery_slider)


btn = widgets.Button(description='Optimize')
display(btn)


btn_analysis = widgets.Button(description='More details')
btn_close_analysis = widgets.Button(description='Close details')


def btn_optimize_eventhandler(obj):

    clear_output()
    display(text, savings_slider,grocery_slider,btn)

    savings = savings_slider.value
    grocery_per_week = grocery_slider.value
    model, variables = create_model(selected_year, savings, grocery_per_week)
    status = model.solve()

    if LpStatus[status] == 'Optimal':
        print("Optimization was successful")
    else:
        valid = widgets.Valid(
            value=False,
            description='Infeasible'        )
        display(valid)
        print("Optimization was unsuccessful. Consider adjusting inputs")
        return
    solution, objective = parse_solution(selected_year,variables,model)
    solved_data = selected_year.join(solution)
    remained_transactions_sum = sum(solved_data['solution']*solved_data['Debit'])
    print(f"Total expenses of proposed retroactive plan: {remained_transactions_sum} EUR")
    #visualize_solution_data(solved_data, grocery_per_week)
    plot_solution_preview(solved_data)
    out = widgets.Output()
    display(out)
    with out:
        display(btn_analysis)
    obj.value = solved_data, out

    
    
def btn_analysis_eventhandler(obj):
    grocery_per_week = grocery_slider.value
    with btn.value[1]:
        clear_output()
        display(btn_close_analysis)
        visualize_solution_data(btn.value[0], grocery_per_week)
        
def btn_close_eventhandler(obj):
    with btn.value[1]:
        clear_output()
        display(btn_analysis)

btn.on_click(btn_optimize_eventhandler)
btn_analysis.on_click(btn_analysis_eventhandler)
btn_close_analysis.on_click(btn_close_eventhandler)


HTML(value='<b>Budget Optimization</b>: Select inputs')

FloatSlider(value=0.3, description='Savings:', max=0.8, step=0.05)

FloatSlider(value=180.0, description='Grocery:', max=500.0, step=5.0)

Button(description='Optimize', style=ButtonStyle())

Valid(value=False, description='Infeasible')

Optimization was unsuccessful. Consider adjusting inputs
